# Natural gradients

This shows some basic usage of the natural gradient optimizer, both on its own and in combination with Adam optimizer.

In [1]:
from matplotlib import pyplot as plt
import warnings
import numpy as np
import tensorflow as tf
import gpflow
from gpflow.test_util import notebook_niter, notebook_range
from gpflow.models import VGP, GPR, SGPR, SVGP
from gpflow.training import NatGradOptimizer, AdamOptimizer, XiSqrtMeanVar

%matplotlib inline
%precision 4
warnings.filterwarnings('ignore')

np.random.seed(0)

N, D = 100, 2
M = 10  # inducing points
X = np.random.uniform(size=(N, D))
Y = np.sin(10 * X)
Z = np.random.uniform(size=(M, D))
adam_learning_rate = 0.01
iterations = 5
    

def make_matern_kernel():
    return gpflow.kernels.Matern52(D)

### "VGP is a GPR"

Below we will demonstrate how natural gradients can turn VGP into GPR in a *single step, if the likelihood is Gaussian*.

Let's start by first creating a standard GPR model with Gaussian likelihood:

In [2]:
gpr = GPR(X, Y, kern=make_matern_kernel())
print('The likelihood of the exact GP model is: %.4f' % gpr.compute_log_likelihood())

The likelihood of the exact GP model is: -231.0899


Now we will create an approximate model which approximates the true posterior via a variational Gaussian distribution. <br\> We initiallize the distribution to be zero mean and unit variance.

In [3]:
vgp = VGP(X, Y, kern=make_matern_kernel(), likelihood=gpflow.likelihoods.Gaussian())
print('The likelihood of the approximate GP model is: %.4f' % vgp.compute_log_likelihood())

The likelihood of the approximate GP model is: -328.8429


Obviously, our initial guess for the variational distribution is not correct, which results in a lower bound to the likelihood of the exact GPR model. We can optimize the variational parameters in order to get a tighter bound. 

In fact, we only need to take **1 step** in the natural gradient direction to recover the exact posterior:

In [4]:
natgrad_optimizer = NatGradOptimizer(gamma=1.)
natgrad_tensor = natgrad_optimizer.make_optimize_tensor(vgp, var_list=[(vgp.q_mu, vgp.q_sqrt)])
session = gpflow.get_default_session()
session.run(natgrad_tensor)
vgp.anchor(session)  # update the cache of the variational parameters in the current session
print('The likelihood of the approximate GP model after a single natgrad step is: %.7f'
      % vgp.compute_log_likelihood())

The likelihood of the approximate GP model after a single natgrad step is: -231.0899982


### Optimize both variational parameters and kernel hyperparameters together

In the Gaussian likelihood case we can iterate between an Adam update for the hyperparameters and a NatGrad update <br\>for the variational parameters. That way, we achieve optimization of hyperparameters as if the model were a GPR.

The trick is to forbid Adam from updating the variational parameters by setting them to not trainable.

In [5]:
# Stop Adam from optimizing the variational parameters
vgp.q_mu.trainable = False
vgp.q_sqrt.trainable = False

# Create Adam tensors for each model
adam_for_vgp_tensor = AdamOptimizer(learning_rate=adam_learning_rate).make_optimize_tensor(vgp)
adam_for_gpr_tensor = AdamOptimizer(learning_rate=adam_learning_rate).make_optimize_tensor(gpr)

natgrad_tensor = NatGradOptimizer(gamma=1.).make_optimize_tensor(vgp, var_list=[(vgp.q_mu, vgp.q_sqrt)])

In [6]:
for i in range(iterations):
    session.run(adam_for_gpr_tensor)
    print('GPR with Adam: iteration %d likelihood %.4f'
          % (i + 1, session.run(gpr.likelihood_tensor)))

gpr.anchor(session)  # update the cache of the parameters in the current session

GPR with Adam: iteration 1 likelihood -230.6706
GPR with Adam: iteration 2 likelihood -230.2508
GPR with Adam: iteration 3 likelihood -229.8303
GPR with Adam: iteration 4 likelihood -229.4093
GPR with Adam: iteration 5 likelihood -228.9876


In [7]:
for i in range(iterations):
    session.run(adam_for_vgp_tensor)
    session.run(natgrad_tensor)
    print('VGP with natural gradients and Adam: iteration %d likelihood %.4f'
          % (i + 1, session.run(vgp.likelihood_tensor)))

# We need to alter their trainable status in order to correctly anchor them in the current session
vgp.q_mu.trainable = True
vgp.q_sqrt.trainable = True
vgp.anchor(session)  # update the cache of the parameters (including the variational) in the current session

VGP with natural gradients and Adam: iteration 1 likelihood -230.6707
VGP with natural gradients and Adam: iteration 2 likelihood -230.2508
VGP with natural gradients and Adam: iteration 3 likelihood -229.8304
VGP with natural gradients and Adam: iteration 4 likelihood -229.4093
VGP with natural gradients and Adam: iteration 5 likelihood -228.9877


Compare GPR and VGP lengthscales after optimisation:

In [8]:
print('GPR lengthscales = %.4f, VGP lengthscales = %.4f'
      % (gpr.kern.lengthscales.value, vgp.kern.lengthscales.value)
)

GPR lengthscales = 0.9686, VGP lengthscales = 0.9686


### Natural gradients also work for the sparse model
Similarly, natural gradients turn SVGP into SGPR in the Gaussian likelihood case. <br\>
We can again combine natural gradients with Adam to update both variational parameters and hyperparameterstoo.<br\>
Here we'll just do a single natural step demonstration.

In [9]:
svgp = SVGP(X, Y, kern=make_matern_kernel(), likelihood=gpflow.likelihoods.Gaussian(), Z=Z)
sgpr = SGPR(X, Y, kern=make_matern_kernel(), Z=Z)

for model in svgp, sgpr:
    model.likelihood.variance = 0.1

In [10]:
print('Analytically optimal sparse model likelihood: %.4f' % sgpr.compute_log_likelihood())

Analytically optimal sparse model likelihood: -281.5616


In [11]:
print('SVGP likelihood before natural gradient step: %.4f' % svgp.compute_log_likelihood())

SVGP likelihood before natural gradient step: -1404.0805


In [12]:
natgrad_tensor = NatGradOptimizer(gamma=1.).make_optimize_tensor(svgp, var_list=[(svgp.q_mu, svgp.q_sqrt)])
session = gpflow.get_default_session()
session.run(natgrad_tensor)
svgp.anchor(session)  # update the cache of the variational parameters in the current session
print('SVGP likelihood after a single natural gradient step:%.4f'
      % svgp.compute_log_likelihood())

SVGP likelihood after a single natural gradient step:-281.5616


### Minibatches
A crucial property of the natural gradient method is that it still works with minibatches.
In practice though, we need to use a smaller gamma.

In [13]:
svgp = SVGP(X, Y, kern=make_matern_kernel(),
            likelihood=gpflow.likelihoods.Gaussian(), Z=Z, minibatch_size=50)
svgp.likelihood.variance = 0.1

natgrad_tensor = \
    NatGradOptimizer(gamma=.1).make_optimize_tensor(svgp, var_list=[(svgp.q_mu, svgp.q_sqrt)])

for _ in range(notebook_niter(100)):
    session.run(natgrad_tensor)

svgp.anchor(session)

In [14]:
likelihood = np.average([svgp.compute_log_likelihood() for _ in notebook_range(1000)])
print('Minibatch SVGP likelihood after NatGrad optimization: %.4f' % likelihood)

Minibatch SVGP likelihood after NatGrad optimization: -282.1565


### Comparison with ordinary gradients in the conjugate case

#### (Take home message: natural gradients are always better)

Compared to SVGP with ordinary gradients with minibatches, the natural gradient optimizer is much faster in the Gaussian case. 

Here we'll do hyperparameter learning together with optimization of the variational parameters, comparing the interleaved natural gradient approach and the one using ordinary gradients for the hyperparameters and variational parameters jointly.

Note that again we need to compromise for smaller gamma value, which we'll keep *fixed* during the optimisation.

In [15]:
svgp_ordinary = SVGP(X, Y, kern=make_matern_kernel(),
                     likelihood=gpflow.likelihoods.Gaussian(), Z=Z, minibatch_size=50)
svgp_natgrad = SVGP(X, Y, kern=make_matern_kernel(),
                    likelihood=gpflow.likelihoods.Gaussian(), Z=Z, minibatch_size=50)

# ordinary gradients with Adam for SVGP
adam_for_svgp_ordinary_tensor = \
    AdamOptimizer(adam_learning_rate).make_optimize_tensor(svgp_ordinary)

# NatGrads and Adam for SVGP
# Stop Adam from optimizing the variational parameters
svgp_natgrad.q_mu.trainable = False
svgp_natgrad.q_sqrt.trainable = False

# Create the optimize_tensors for SVGP
adam_for_svgp_natgrad_tensor = \
    AdamOptimizer(adam_learning_rate).make_optimize_tensor(svgp_natgrad)

natgrad_tensor = \
    NatGradOptimizer(gamma=.1).make_optimize_tensor(
        svgp_natgrad, var_list=[(svgp_natgrad.q_mu, svgp_natgrad.q_sqrt)]
    )

Let's optimise the models now:

In [16]:
# Optimize svgp_ordinary
for _ in range(notebook_niter(100)):
    session.run(adam_for_svgp_ordinary_tensor)

svgp_ordinary.anchor(session)

# Optimize svgp_natgrad
for _ in range(notebook_niter(100)):
    session.run(adam_for_svgp_natgrad_tensor)
    session.run(natgrad_tensor)

svgp_natgrad.anchor(session)

In [17]:
likelihood = np.average([svgp_ordinary.compute_log_likelihood() for _ in notebook_range(1000)])
print('SVGP likelihood after ordinary Adam optimization: %.4f' % likelihood)

SVGP likelihood after ordinary Adam optimization: -207.4113


In [18]:
likelihood = np.average([svgp_natgrad.compute_log_likelihood() for _ in notebook_range(1000)])
print('SVGP likelihood after NatGrad and Adam optimization: %.4f' % likelihood)

SVGP likelihood after NatGrad and Adam optimization: -197.0323


### Comparison with ordinary gradients in the non-conjugate case

#### (Take home message: natural gradients are usually better)

We can use nat grads even when the likelihood isn't Gaussian. It isn't guaranteed to be better, but it usually is better in practical situations.

In [19]:
Y_binary = np.random.choice([1., -1], size=X.shape)

vgp_bernoulli = VGP(X, Y_binary, kern=make_matern_kernel(), likelihood=gpflow.likelihoods.Bernoulli())
vgp_bernoulli_natgrad = VGP(X, Y_binary, kern=make_matern_kernel(), likelihood=gpflow.likelihoods.Bernoulli())

# ordinary gradients with Adam for VGP with Bernoulli likelihood
adam_for_vgp_bernoulli_tensor = \
    AdamOptimizer(adam_learning_rate).make_optimize_tensor(vgp_bernoulli)

# NatGrads and Adam for VGP with Bernoulli likelihood
# Stop Adam from optimizing the variational parameters
vgp_bernoulli_natgrad.q_mu.trainable = False
vgp_bernoulli_natgrad.q_sqrt.trainable = False

# Create the optimize_tensors for VGP with natural gradients
adam_for_vgp_bernoulli_natgrad_tensor = \
    AdamOptimizer(adam_learning_rate).make_optimize_tensor(vgp_bernoulli_natgrad)

natgrad_tensor = \
    NatGradOptimizer(gamma=.1).make_optimize_tensor(
        vgp_bernoulli_natgrad,
        var_list=[(vgp_bernoulli_natgrad.q_mu, vgp_bernoulli_natgrad.q_sqrt)]
    )

In [20]:
# Optimize vgp_bernoulli
for _ in range(notebook_niter(100)):
    session.run(adam_for_vgp_bernoulli_tensor)

vgp_bernoulli.anchor(session)

# Optimize vgp_bernoulli_natgrad
for _ in range(notebook_niter(100)):
    session.run(adam_for_vgp_bernoulli_natgrad_tensor)
    session.run(natgrad_tensor)

vgp_bernoulli_natgrad.anchor(session)

In [21]:
print('VGP likelihood after ordinary Adam optimization: %.4f' % vgp_bernoulli.compute_log_likelihood())

VGP likelihood after ordinary Adam optimization: -146.1205


In [22]:
print('VGP likelihood after NatGrad + Adam optimization: %.4f'
      % vgp_bernoulli_natgrad.compute_log_likelihood())

VGP likelihood after NatGrad + Adam optimization: -143.9415


We can also choose to run natural gradients in another parameterization.<br\>
The 
sensible choice is the model parameters (q_mu, q_sqrt), which is already in gpflow.

In [23]:
vgp_bernoulli_natgrads_xi = VGP(X, Y_binary,
                                kern=make_matern_kernel(), likelihood=gpflow.likelihoods.Bernoulli())

var_list = [(vgp_bernoulli_natgrads_xi.q_mu, vgp_bernoulli_natgrads_xi.q_sqrt, XiSqrtMeanVar())]

# Stop Adam from optimizing the variational parameters
vgp_bernoulli_natgrads_xi.q_mu.trainable = False
vgp_bernoulli_natgrads_xi.q_sqrt.trainable = False

# Create the optimize_tensors for VGP with Bernoulli likelihood
adam_for_vgp_bernoulli_natgrads_xi_tensor = \
    AdamOptimizer(adam_learning_rate).make_optimize_tensor(vgp_bernoulli_natgrads_xi)

natgrad_tensor = \
    NatGradOptimizer(gamma=.01).make_optimize_tensor(
        vgp_bernoulli_natgrads_xi,
        var_list=var_list
    )

In [24]:
# Optimize vgp_bernoulli_natgrads_xi
for _ in range(notebook_niter(100)):
    session.run(adam_for_vgp_bernoulli_natgrads_xi_tensor)
    session.run(natgrad_tensor)

vgp_bernoulli_natgrads_xi.anchor(session)

In [25]:
print('VGP likelihood after NatGrads with XiSqrtMeanVar + Adam optimization: %.4f'
      % vgp_bernoulli_natgrads_xi.compute_log_likelihood())

VGP likelihood after NatGrads with XiSqrtMeanVar + Adam optimization: -143.9024


With sufficiently small steps, it shouldn't make a difference which transform is used, but for large 
steps this can make a difference in practice.